# Image Sampler from Camera

## Import

In [1]:
import time
import cv2
import matplotlib.pyplot as plt
from datetime import datetime
from pseyepy import Camera, Stream


In [ ]:
print(cv2.getBuildInformation())

## Save Path

In [ ]:
save_path = "./footage_test/"
output_json_path = "./coordinates_data.json"

## Sample Image

In [ ]:
# initialize all connected cameras
c = Camera(fps=100)

buffer = []
max_count = 200
# read from the camera/s
while True:
    frames, timestamp = c.read()
    if frames and timestamp:
        buffer.append(zip(frames,timestamp))
    # print(len(buffer))
    if len(buffer)>=max_count:break
c.end()

for buff in buffer:
    for i,(frame,ts) in enumerate(buff):
    # for i,(frame,ts) in enumerate(zip(frames,timestamp)):
        time = datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H:%M:%S.%f')
        #print(i, time)
        # plt.figure()
        # plt.imshow(frame)
        #print(f'{save_path}{i}/{time}.png')
        cv2.imwrite(f'{save_path}{i}/{time}.png',frame)
    # when finished, close the camera

about to init
pyc count: 
4


In [6]:
import cv2
import numpy as np

# Path to the video file
video_path = "./footage_video/calibration1_0.avi"

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# # Define a kernel for the morphological operation
# kernel = np.ones((3, 3), np.uint8)
kernel = np.array([[0, 1, 0],[1, 1, 1],[0, 1, 0]], np.uint8)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Read until the video is completed
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret:
            # Convert frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Apply binarization
            _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

            # Apply the closing morphological operation
            opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

            # Display the resulting frame
            cv2.imshow('Frame', opening)
            # cv2.imshow('Frame', binary)

            # Press Q on keyboard to exit
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        else:
            break

    # When everything done, release the video capture object
    cap.release()

    # Close all the frames
    cv2.destroyAllWindows()

In [10]:
import cv2
import numpy as np
import json

# Path to the video file
video_path = "./footage_video/calibration1_0.avi"

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Define a kernel for the morphological operation
kernel = np.ones((2, 2), np.uint8)

# Initialize a list to store the coordinates
coordinates_data = []

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    frame_number = 0
    # Read until the video is completed
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret:
            frame_number += 1
            # Convert frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Apply binarization
            _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

            # Apply the closing morphological operation
            closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

            # Find contours
            contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Initialize a list to store coordinates for the current frame
            frame_coordinates = []

            # Convert the frame to RGB
            closing = cv2.cvtColor(closing, cv2.COLOR_GRAY2BGR)

            # Loop over the contours to find the center coordinates
            for contour in contours:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    # Draw the center of the contour on the frame
                    cv2.circle(closing, (cX, cY), 1, (0, 255, 0), -1)
                    # Append the coordinates to the list
                    frame_coordinates.append((cX, cY))

            # Append the frame data to the main list
            coordinates_data.append({
                "frame": frame_number,
                "coordinates": frame_coordinates
            })

            # Display the resulting frame
            cv2.imshow('Frame', closing)

            # Press Q on keyboard to exit
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        else:
            break

    # When everything done, release the video capture object
    cap.release()

    # Close all the frames
    cv2.destroyAllWindows()

output_json_path = f"./coordinates_data_{video_path.split('/')[-1][:-4]}.json"
# Save the collected coordinates data to a JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(coordinates_data, json_file, indent=4)

print(f"Coordinates data saved to {output_json_path}")

Coordinates data saved to ./coordinates_data_calibration1_0.json
